
![picTitle](../logo_uadb_o.png)
## "Financial Inclusion in Africa from Zindi Africa"
# LE CHALENGE C'EST QUOI ? 😎😎🙄🙄
Lien vers la source https://zindi.africa/competitions/inclusion-financiere-en-afrique/data

#### Il vous est demandé de prédire la probabilité que la personne ait un compte bancaire ou non (Oui = 1, Non = 0), pour chaque identifiant unique dans l’ensemble de données de test. Vous formerez votre modèle sur 70% des données et testerez votre modèle sur les 30% dernières données, dans quatre pays d’Afrique de l’Est – le Kenya, le Rwanda, la Tanzanie et l’Ouganda.

# On y va !

#### La première étape consiste à séparer les variables indépendantes et la cible (bank_account) des données de formation. Ensuite, on transforme les valeurs cibles du type de données objet en valeurs numériques en utilisant [LabelEncoder] (https://towardsdatascience.com/categorical-encoding-using-label-encoding-and-one-hot-encoder-911ef77fb5bd).

## 1. On importe les modules et librairies necessaires

In [35]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

# 2. On charge les données

In [9]:
# charger les données
train = pd.read_csv('../data/Train.csv')
test = pd.read_csv('../data/Test.csv')
ss = pd.read_csv('../data/SubmissionFile.csv')
variables = pd.read_csv('../data/VariableDefinitions.csv')

In [12]:
# Inspecter les données de train
train.head(3)

,country,year,uniqueid,bank_account,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,Kenya,2018,uniqueid_1,Yes,Rural,Yes,3,24,Female,Spouse,Married/Living together,Secondary education,Self employed
1,Kenya,2018,uniqueid_2,No,Rural,No,5,70,Female,Head of Household,Widowed,No formal education,Government Dependent
2,Kenya,2018,uniqueid_3,Yes,Urban,Yes,5,26,Male,Other relative,Single/Never Married,Vocational/Specialised training,Self employed


In [13]:
# Verifie les fichiers manquants
print('Valeurs manquants:', train.isnull().sum())

Valeurs manquants: country                   0
year                      0
uniqueid                  0
bank_account              0
location_type             0
cellphone_access          0
household_size            0
age_of_respondent         0
gender_of_respondent      0
relationship_with_head    0
marital_status            0
education_level           0
job_type                  0
dtype: int64


##### On constate qu'il ya pas de donnés manquants

## 3. Preparation des données
##### Avant de former le modèle de prédiction, vous devez effectuer un [nettoyage] (https://www.dataquest.io/blog/machine-learning-preparing-data/) et un [prétraitement] (https://towardsdatascience.com/data-preprocessing-concepts-fa946d11c825) des données. Il s'agit d'une étape très importante ; votre modèle ne sera pas performant sans ces étapes.

### On effectiue un fitting sur les données les plus pertinents du sample fourni

In [17]:
features = ['job_type', 'cellphone_access', 'relationship_with_head', 'education_level', 
            'household_size', 'location_type', 'country']

### On creer une fonction qui fait du preprocessing sur les données fournies

In [23]:
def preparer_les_donnees(file_path):
    data = pd.read_csv(file_path)

    le = LabelEncoder()
    if 'bank_account' in data:
        # Pour le cas ou les données fournis sont les données d'entrainement
        data['bank_account'] = le.fit_transform(data['bank_account'])
    data['job_type'] = le.fit_transform(data['job_type'])

    data['cellphone_access'] = le.fit_transform(data['cellphone_access'])

    data['gender_of_respondent'] = le.fit_transform(data['gender_of_respondent'])

    data['relationship_with_head'] = le.fit_transform(data['relationship_with_head'])

    data['education_level'] = le.fit_transform(data['education_level'])

    data['marital_status'] = le.fit_transform(data['marital_status'])
    data['country'] = le.fit_transform(data['country'])

    data['location_type'] = le.fit_transform(data['location_type'])

    return data

## 4. Build de notre modèle

##### Une partie de l'ensemble de données d'entraînement sera utilisée pour évaluer nos modèles et trouver celui qui donne les meilleurs résultats avant de l'utiliser dans l'ensemble de données de test.

In [20]:
#  cette fonction separe les donnes de test et de train
def split_data_and_target(data):
    dataset = data[features]
    target = data['bank_account']

    return dataset, target

##  On utilise **SVM** dans notre cas

In [31]:
def appliquer_le_svm(data, target):
    X_train, x_test, Y_train, y_test = train_test_split(data, target, test_size=0.3)
    svc = SVC(gamma='auto', kernel='rbf', C=5)
    svc.fit(X_train, Y_train)
    pred = svc.predict(x_test)
    print("SVM: Accuracy...", accuracy_score(y_test, pred))
    print("SVM: Mean Accuracy Error : ", 1 - accuracy_score(y_test, pred))
    return svc

#### Seuls 30% de l'ensemble de données de formation seront utilisés pour évaluer les modèles. Le paramètre 'stratify = y_target_train' assurera un équilibre égal des valeurs des deux classes ('oui' et 'non') pour les ensembles de formation et de validation.

## 5. Evalution et prédiction

#### Cette fonction **evaluate_le_model_pour_submission** utilise le model apres nettoyage des données et effectue une prediction


In [36]:
def evaluate_le_model_pour_submission(model):
    submission_data = preparer_les_donnees('../data/Test.csv')
    filtered_data = submission_data[features]
    submission_pred = model.predict(filtered_data)    
    return submission_pred

In [26]:
def exporter_submission_data(predicted):
    test = pd.read_csv('../data/Test.csv')
    test['uniqueid'] = test['uniqueid'] + ' x ' + test['country']

    submission = pd.DataFrame(columns=['uniqueid', 'bank_account'], data=test)
    submission.bank_account = predicted

    submission.to_csv('../soumettre/SubmissionFile.csv', index=False)

#### Cette fonction montre le format de soumission pour ce challenge, avec la colonne "uniqueID" reflétant celle de "Test.csv" et la colonne "bank_account" contenant nos prédictions. 
#### L’identifiant de la variable dans le fichier de soumission est: uniqueid x country name.

## TEST

In [27]:
def __main__():
    result_data = preparer_les_donnees('../data/Train.csv')  # data
    data, target = split_data_and_target(result_data)
    model = appliquer_le_svm(data, target)    
    predicted = evaluate_le_model_pour_submission(model)
    exporter_submission_data(predicted)

# RUN 😎

In [32]:
__main__()

SVM: Accuracy..   0.8809861150467555
SVM: Main accuracy error :   0.1190138849532445
